Pre-processing for data cleaning 어떤 스토어들은 return 때문에 하루 매출 자체가 음수가 되는 경우도 있음.

음수들은 Variance를 증대시키므로 prediction interval(http://bit.ly/2DnkoFD)

이 커지게 되어 prediction에 악영향을 줄 수 있음.

이 음수들을 제거하기 위해 카드 아이디를 이용하여 같은 카드의 negative transaction의 시간보다 작으면서 가장 최근 transaction 중

1) 절대값이 같은 양수 가격의 transaction 을 제거 혹은

2) 절대값이 더 큰 양수 가격의 transaction 에서 음수 transaction의 가격을 빼는 방법으로 음수들을 제거.

만약 해당 카드 아이디가 그 전에 존재하지 않거나 1, 2에 해당하는 경우가 없을 경우 해당 환불의 실제 구매는 데이터의 시작 시간보다 과거이므로 데이터에서 제외 (line 32 - 66).

business hour가 0시에 꼭 끝나지 않음으로 스토어에 따라 시간을 조정하여 하루를 1 business day로 만들면 좋음. business hour가 각 스토어마다 다르고, 스토어의 숫자가 많기도 하고 downsampling (크리스알본 : http://bit.ly/37A5l95 ) 하는 기간이 길면 이 부분은 어느정도 해결이 되므로 이 부분은 하지 않기로 결정.

판매금액과 로그를 씌운 판매금액이 완전히는 아니지만 어느정도는 정규분포에 비슷함을 QQ plot으로 확인. outlier들의 영향을 줄이기 위해 로그를 씌운 판매금액을 종속변수로 사용하기로 결정.

log를 씌운 후 값이 null이 되거나 무한이 되면 데이터셋에서 제거.

null/무한대인 값들이 몇개인지 세어 몇 퍼센트의 샘플이 null/무한대인지 계산 후 최종 prediction에서 (1 - probability of no sales) 값으로 예측값을 곱하여 매출 예상에서 판매가 없을 확률만큼 discount함 (line 301).

PACF (편자기상관함수 http://bit.ly/2OeRail)

를 통해 AR(0), AR(1), AR(2)를 확인. 하지만 AIC를 통해 ARIMA모델에 사용될 pqr 값들을 찾을 때 0~2의 값들을 통해 구한 pqr보다 0~1 사이로 찾은 pqr이 더 정확한 예측을 했기 때문에 pqr의 범위를 0~1로 설정 (line 141-160)

스토어마다 데이터 기간이 다름. 따라서 스토어 마다 다른 리샘플링 기간을 줄 필요가 있음.

샘플이 충분하다면 최대한의 기간을 한 묶음으로 묶어 최소의 로 필요한 기간만큼 판매예측 하는 것이 더 정확한 예측 가능할 것이라 판단.

요일에 따른 seasonality가 있으므로 최소 1주 단위로 묶어 seasonality를 없앨 필요가 있음.

매년 있는 seasonality 또한 있으므로 365.25를 나눠서 잘 떨어지는 숫자로 정할 필요가 있음. 365.25를 나누면 7 일경우 52.18, 14일경우 26.09, 17.39, 28의 경우 13.04. 잘 나누어 떨어지는 숫자인 28, 14, 7일 단위로로 downsampling 하여, 최소 샘플의 숫자를 넘지 못하는 경우 점점 적은 기간으로 downsampling 함 (line 309-316).

리샘플링 할 때 28 일씩 할 경우 일별로 묶인 샘플의 숫자로 28로 나누어 떨어지지 않으면 마지막 bin 안에 들어있는 샘플의 숫자가 다른 묶음안의 숫자보다 작음.

각각의 bin 에 해당하는 기간이 같아야하고, 최근의 데이터가 먼 과거의 데이터보다 더 중요하기 때문에 28일로 나누어 떨어지지 않는 경우 앞의 남는 샘플들을 제거. 14일 단위의 bin으로 묶을 때도 같은 방법을 사용 (line 267).

ARIMA모델에 사용되는 pqr 값을을 찾을때는 overfitting을 방지하기 위해 AIC 값을 이용해서 찾게 만듬 (parsimony) (line 145 - 160).

산식 E(return) == 0.13 (100 + x) / 365 p = alpha p E(loss) == 1 (1-p) == beta (1-p) E(total return) == E(return) + E(loss) finding optimal p: alpha p == beta (1 - p) alpha p - beta p == beta p == beta / (alpha - beta) p == optimal p LN 세일즈가 표준분포에 흡사하세 퍼져있기 때문에 optimal p를 사용하여 optimal Z score를 계산. 이 Z score와 STD를 사용하여 확률적으로 점수계산공식에 가장 이상적인 예측값을 계산. (line 279 - 288) Optimization process: 28일, 14일, 7일씩 downsampling하여 예측하면 향후 84일 (28 3) 혹은 98일 (14 7 or 7 14) 동안의 매출을 예측 가능. 위의 E(return)안의 x값을 늘려 optimal p를 조절하는 방식으로 예측에서 빠진 기간동안의 매출을 채우기로 결정. 28일씩 리샘플링한 경우는 (84일 예측) x==22.8, 14일씩 리샘플링한 경우는 (98일 예측) x == 30 (30 이상은 test 해보지 않음)으로 했을 경우 좋은 결과가 나왔음 (Score == 49.42492). 최소의 샘플 숫자는 4, 5, 6, 8 중 6의 결과가 가장 좋아 6으로 결정.

In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import math
import seaborn as sns
import warnings
import statsmodels.api as sm
import matplotlib.pyplot as plt
import itertools
import scipy.stats as st
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")  # specify to ignore warning messages

sns.set(color_codes=True)

path = './data/'

test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'submission.csv')

df_copy = test.copy()
df_copy.date = pd.to_datetime(df_copy.date)

# df_copy.date = pd.to_datetime(df_copy.date + " " + df_copy.time, format='%d/%m/%y %H:%M:%S')

df_copy.date = pd.to_datetime(df_copy.date.astype(str) + " " + df_copy.time, format='%Y-%m-%d %H:%M:%S')

# Remove negative values from the data set.

def reduce_noise_by_removing_neg_vals(df_copy):
    df_pos = df_copy[df_copy.amount > 0]
    df_neg = df_copy[df_copy.amount < 0]

    start = datetime.now()

    for nega_i in df_neg.to_records()[:]:
        store_i = nega_i[1]
        date_i = nega_i[2]
        card_i = nega_i[4]
        amt_i = nega_i[5]
        row_i = df_pos[df_pos.store_id == store_i]
        row_i = row_i[row_i.card_id == card_i]
        row_i = row_i[row_i.amount >= abs(amt_i)]
        row_i = row_i[row_i.date <= date_i]
        if len(row_i[row_i.amount == abs(amt_i)]) > 0:
            row_i = row_i[row_i.amount == abs(amt_i)]
            matched_row = row_i[row_i.date == max(row_i.date)]
            # df_pos.loc[matched_row.index, 'amount'] = 0
            df_pos = df_pos.loc[~df_pos.index.isin(matched_row.index), :]
        elif len(row_i[row_i.amount > abs(amt_i)]) > 0:
            matched_row = row_i[row_i.date == max(row_i.date)]
            df_pos.loc[matched_row.index, 'amount'] = matched_row.amount + amt_i
        # else:
        #     pass
            # no_match.append(nega_i)
    end = datetime.now()
    time_took = (end - start).seconds / 60

    print(round(time_took, 2))
    return df_pos

df_pos = reduce_noise_by_removing_neg_vals(df_copy)

def adf_test(y):
    # perform Augmented Dickey Fuller test
    print('Results of Augmented Dickey-Fuller test:')
    dftest = adfuller(y, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value ({})'.format(key)] = value
    print(dfoutput)

def ts_diagnostics(y, lags=None, title='', filename=''):
    '''
    Calculate acf, pacf, qq plot and Augmented Dickey Fuller test for a given time series
    '''
    if not isinstance(y, pd.Series):
        y = pd.Series(y)

    # weekly moving averages (5 day window because of workdays)
    rolling_mean = pd.Series.rolling(y, window=2).mean()
    rolling_std = pd.Series.rolling(y, window=2).std()

    fig = plt.figure(figsize=(14, 12))
    layout = (3, 2)
    ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
    acf_ax = plt.subplot2grid(layout, (1, 0))
    pacf_ax = plt.subplot2grid(layout, (1, 1))
    qq_ax = plt.subplot2grid(layout, (2, 0))
    hist_ax = plt.subplot2grid(layout, (2, 1))

    # time series plot
    y.plot(ax=ts_ax)
    rolling_mean.plot(ax=ts_ax, color='crimson')
    rolling_std.plot(ax=ts_ax, color='darkslateblue')
    plt.legend(loc='best')
    ts_ax.set_title(title, fontsize=24)

    # acf and pacf
    plot_acf(y, lags=lags, ax=acf_ax, alpha=0.5)
    plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.5)

    # qq plot
    sm.qqplot(y, line='s', ax=qq_ax)
    qq_ax.set_title('QQ Plot')

    # hist plot
    y.plot(ax=hist_ax, kind='hist', bins=25)
    hist_ax.set_title('Histogram')
    plt.tight_layout()
    plt.show()

    # perform Augmented Dickey Fuller test
    print('Results of Dickey-Fuller test:')
    dftest = adfuller(y, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['test statistic', 'p-value', '# of lags', '# of observations'])
    for key, value in dftest[4].items():
        dfoutput['Critical Value (%s)' % key] = value
    print(dfoutput)
    return

df = df_pos.copy()
test_groupby_date_store = df.groupby(['date', 'store_id'])['amount', 'holyday'].sum()
test_groupby_date_store = test_groupby_date_store.reset_index()

test_groupby_date_store = test_groupby_date_store.set_index('date')
store_list = test_groupby_date_store.store_id.unique()

store_list.sort()


def get_optimal_params(y):
    # Define the p, d and q parameters to take any value between 0 and 1

    param_dict = {}
    for param in pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(y,
                                            order=param,
                                            )
            results = mod.fit()
            model = ARIMA(y, order=param)
            results_ARIMA = model.fit(disp=-1)
            results_ARIMA.summary()
            param_dict[results.aic] = param
        except:
            continue

    min_aic = min(param_dict.keys())
    optimal_params = param_dict[min_aic]
    return optimal_params


sampling_p = 28
mean_period = 2 * 3 #14 * 2*3

predic_len = math.floor(100 / sampling_p)

expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365
expected_loss_pct_lending = 1.00
optimal_prob = expected_loss_pct_lending / (expected_loss_pct_lending + expected_return_pct_lending)
optimal_z_score = st.norm.ppf(optimal_prob)

min_period = 6


max_pdq = 2
p = d = q = range(0, max_pdq)
pdq = list(itertools.product(p, d, q))


pdqs = dict()
print(optimal_prob)
print(optimal_z_score)
output_file_name_fmt = 'py_4arima_pos_sep_{optimal_p}-{sampling_period}_no_sales_prob&no mean{mean_period}&min_period {min_period}_pdq{max_pdq}.csv'
output_file_name = output_file_name_fmt.format(optimal_p=round(optimal_prob, 4),
                                               sampling_period=sampling_p,
                                               mean_period=mean_period,
                                               min_period=min_period,
                                               max_pdq=max_pdq)

submission_copy = submission.copy()

def arima_main(input_df, sampling_period_days, fcst_period):
    input_df = input_df[len(input_df) % sampling_period_days:].resample(str(sampling_period_days) + 'D').sum()
    prob_of_no_sales = len(input_df[(input_df.amount == 0) | (input_df.amount.isna())]) / len(input_df)
    ts_log = np.log(input_df.amount)
    ts_log = ts_log[~ts_log.isin([np.nan, np.inf, -np.inf])]

    if len(ts_log) < min_period:
        return None
    if sampling_period_days >= 28:
        expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365
    elif sampling_period_days >= 14:
        expected_return_pct_lending = 0.13 * (100 + 16 + 14) / 365
    else:
        expected_return_pct_lending = 0.13 * (100 + 16 + 6.8) / 365

    expected_loss_pct_lending = 1.00
    optimal_prob = expected_loss_pct_lending / (expected_loss_pct_lending + expected_return_pct_lending)
    optimal_z_score = st.norm.ppf(optimal_prob)

    optimal_params = get_optimal_params(ts_log)
    pdqs[store_i] = optimal_params

    model = ARIMA(ts_log, order=optimal_params)
    results_ARIMA = model.fit(disp=-1)
    fcst = results_ARIMA.forecast(fcst_period)

    fcst_means = fcst[0]
    fcst_stds = fcst[1]
    fcst_i = fcst_means - (fcst_stds * optimal_z_score)
    fcst_i = sum(map(lambda x: np.exp(x) if np.exp(x) > 0 else 0, fcst_i))
    prediction_i = fcst_i * (1 - prob_of_no_sales)
    return prediction_i

for store_i in store_list[:]:
    prediction_i = None
    test_df = test_groupby_date_store[test_groupby_date_store.store_id == store_i]
    test_df_daily = test_df.resample('D').sum()
    prediction_i = arima_main(test_df_daily, sampling_period_days=28, fcst_period=3)
    # if prediction_i is None:
    #     prediction_i = arima_main(test_df_daily, sampling_period_days=21, fcst_period=4)
    if prediction_i is None:
        prediction_i = arima_main(test_df_daily, sampling_period_days=14, fcst_period=7)
    if prediction_i is None:
        prediction_i = arima_main(test_df_daily, sampling_period_days=7, fcst_period=12)
    if prediction_i is None:
        test_df = test_df_daily[len(test_df_daily) % 14:].resample('14D').sum()

        prob_of_no_sales = len(test_df[(test_df.amount == 0) | (test_df.amount.isna())]) / len(test_df)
        ts_log = ts_log[~ts_log.isin([np.nan, np.inf, -np.inf])]
        ts_log_wkly = np.log(test_df.amount)

        estimated_amt = np.exp(ts_log_wkly.mean() - ts_log_wkly.std() * optimal_z_score) * (1 - prob_of_no_sales)
        prediction_i = estimated_amt * mean_period

    submission_copy.loc[submission_copy['store_id'] == store_i, 'total_sales'] = prediction_i

submission_copy.to_csv(output_file_name, index=False)

print(output_file_name)

5.38
0.9580957780787686
1.7290036387221377
py_4arima_pos_sep_0.9581-28_no_sales_prob&no mean6&min_period 6_pdq2.csv
